# causalgraph advanced add/remove

The tutorial describes the advanced add/remove functionalities of the [causalgraph](https://gitlab.cc-asp.fraunhofer.de/kausalassist/causal-services/causalgraph) module.

Required packages to run this tutorial are:
- jupyter (to run this notebook)
- [owlready2 0.35](https://owlready2.readthedocs.io/en/v0.35/) (backend for causalgraph store)


### Beneficial prior knowledge
causalgraph (cg) relies on semantic technology, mainly through the python package 'owlready2'.
The core functions are already implemented in owlready2 and wrapped in cg to avoid misuse.

Therefore the author advises to look at the tool [owlready2](https://owlready2.readthedocs.io/en/v0.35/) and get to know some essential functions of it. Especially the tutorial on [creating individuals](https://owlready2.readthedocs.io/en/v0.35/class.html) is an excellent basic knowledge to get the inner workings of cg.

Furthermore, the tutorial 01-Basics_of_causalgraph helps.

### Covered topics in this tutorial

Functions introduced:
- add causal_edges with confidence and timelag
- remove nodes


#### CausalNodes and CausalEdges
The most important concept for this tutorial is the relation between [*CausalNodes*](https://kausalassist.pages.fraunhofer.de/causal-services/causalgraph-ontology/#CausalNode) and [*CausalEdges*](https://kausalassist.pages.fraunhofer.de/causal-services/causalgraph-ontology/#CausalEdge). 

##### CausalNodes
A CausalNode can either represent an effect, a cause or both (e.g. effect of Node1 and cause for Node3). CausalNodes are no causally connected with a direct link, but through an Individual of Type *CausalEdge*:

##### CausalEdges
The CausalEdge is introduced to hold further information about the causal connection. E.g.:
- *Confidence* (in it's existence), 
- *Creator* (Human Expert or ML-Algorithm), 
- *TimeLag in seconds* between Cause and Effect,
- ... 

This is modeled similarily to this picture in *causalgraph-ontology* (see documentation of [*CausalEdges*](https://kausalassist.pages.fraunhofer.de/causal-services/causalgraph-ontology/#CausalEdge)):

![](https://mermaid.ink/svg/eyJjb2RlIjoiZ3JhcGggVERcbiAgICBzdWJncmFwaCBDYXVzYWwgU3RydWN0dXJlXG4gICAgICAgIEVkZ2V7Q2F1c2FsRWRnZX0gLS0-IHxoYXNfY2F1c2V8IE5vZGUxKENhdXNhbE5vZGUpXG4gICAgICAgIEVkZ2UgLS0-IHxoYXNfZWZmZWN0fCBOb2RlMihDYXVzYWxOb2RlKVxuICAgIGVuZCBcbiAgICBzdWJncmFwaCBBZGRpdG9uYWwgQ2F1c2FsIEVkZ2UgSW5mb3JtYXRpb25cbiAgICAgICAgRWRnZSAtLT4gfGhhc19jcmVhdG9yfCBDcmVhdG9yWy4uLkNyZWF0b3JdXG4gICAgICAgIEVkZ2UgLS0-IHxoYXNfdGltZV9sYWd8IFRpbWVsYWdbLi4uVGltZWxhZ11cbiAgICBlbmRcbiAgICAiLCJtZXJtYWlkIjp7InRoZW1lIjoiZGVmYXVsdCJ9LCJ1cGRhdGVFZGl0b3IiOmZhbHNlLCJhdXRvU3luYyI6dHJ1ZSwidXBkYXRlRGlhZ3JhbSI6ZmFsc2V9)



## Working with the 'cg' module

### Loading the module
The facade class for the module is 'Graph' which is loaded below.

In [ ]:
# add the current path to the sys.path to find 'causalgraph.Graph'
import os
import sys
import time
sys.path.insert(1, os.path.join(sys.path[0], '..'))
# causalgraph imports
from causalgraph import Graph
import causalgraph.utils.owlready2_utils as owl2utils
from causalgraph.utils.path_utils import get_project_root
print(get_project_root())

### Instantiate the 'Graph'

During instantiation, the '**Graph**' loads the class-structure-defining ontology ('onto' in the examples at [owlready2-docs](https://owlready2.readthedocs.io/en/v0.35/onto.html)) into **Graph.store**. 

This ontology and the individuals created from it are saved in sqlite3 quadstore, which is linked by attribute **Graph.world**. To read more about an owlready2.World please refer to ['owlready2.World'](https://owlready2.readthedocs.io/en/v0.35/world.html).

In [ ]:
# Specify new sqlite db for this example
# delete old SQL-DB if exists (restart of kernel may be necessary):
sql_file_name='example1.sqlite3'
if os.path.exists(sql_file_name):
    os.remove(sql_file_name)
    print(f"Deleted old db with name {sql_file_name}")

#### Add CausalNodes and CausalEdges

All the ```Add``` functions are grouped in the Object ```Graph.add```. Therefore one can add CausalEdges and CausalNodes in the following way:

In [ ]:
# init graph
G = Graph(sql_db_filename=sql_file_name)
# add nodes
G.add.causal_node("cause")
G.add.causal_node("effect")
# add edge
G.add.causal_edge("cause", "effect", "causal_edge")
# show graph
G.draw.nx()

### Advanced adding 

Timelag and confidence can be added, e.g. shown with the following:

In [ ]:
# Advanced calling with confidence and time_lag. Comments can be added as well
# Add two CausalNodes and show these individuals in graph
G.add.causal_node("another_effect")
G.add.causal_edge("effect", "another_effect", confidence=0.2, time_lag_s=1.5, comment=["An edge between effect and another effect."])
G.draw.nx()

#### Adding a creator node
A Creator can be added with the following:

In [ ]:
# Add creator 'master_creator' 
creator_node_name = G.add.individual_of_type("Creator", name_for_individual="master_creator")
creator_node = G.get_entity(creator_node_name)
print(creator_node_name)

# Add creator 'master_creator' to causal_node and causal_edge
G.add.causal_node("nodewithcreator", hasCreator=[creator_node])
new_edge = G.add.causal_edge("effect", "another_effect", "new_edge", confidence=0.2, time_lag_s=1.5, hasCreator=[creator_node])
G.draw.nx()
# Inspect properties of new_edge
inspected_node = G.get_entity(new_edge)
for prop in inspected_node.get_properties():
    for value in prop[inspected_node]:
        print(".%s == %s" % (prop.python_name, value))

# Remove CausalNodes and CausalEdges
All the 'Remove' functions are grouped in the Object *Graph.remove*. Therefore one can remove CausalEdges and CausalNodes as follows:

remove causal nodes

In [ ]:
# Remove two CausalNodes and show the rest of individuals:
print("Graph before remove")
print(f"Now we have these individuals:\n {list(G.store.individuals())}")
G.draw.nx()
G.remove.causal_node("nodewithcreator")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

remove causal edges 

In [ ]:
# Remove one edge and show the rest of individuals:
print("Graph before remove")
G.draw.nx()
G.remove.causal_edge_by_name("causal_edge")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

remove all causal edges between two causal nodes

In [ ]:
# Remove one edge, between two nodes and show the rest of individuals:
## adds two edges between 'cause' and 'effect'
G.add.causal_edge("cause", "effect", "edge_1")
G.add.causal_edge("effect", "cause", "edge_2")
G.add.causal_edge("cause", "another_effect", "edge_3")
G.add.causal_edge("effect", "another_effect", "edge_4")
print("Graph before remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")
## deletes all edges between 'cause' and 'effect'.
G.remove.causal_edges("cause", "effect")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

remove all causalEdges which are connected with one causalNode

In [ ]:
# Remove one edge, between two nodes and show the rest of individuals:
## adds two edges between 'cause' and 'effect'
G.add.causal_node("another_cause")
G.add.causal_edge("cause", "effect", "edge_5")
G.add.causal_edge("effect", "cause", "edge_6")
G.add.causal_edge("another_effect", "another_cause", "edge_7")
print("Graph before remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")
## deletes all edges that are connected with 'effect' 
G.remove.causal_edges_from_node("effect")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

#### Removing a creator node
A Creator can be removed with the following:

In [ ]:
# Add creator 'test_creator' 
creator_node_name = G.add.individual_of_type("Creator", "test_creator")
print(f"Now, we have these individuals:\n {list(G.store.individuals())}")
# Delete creator 'test_creator' 
G.remove.delete_individual_of_type("test_creator", 'Creator')
print(f"Now, we have these individuals:\n {list(G.store.individuals())}")